In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from glasses.nn.models.classification.fishnet import *
from glasses.nn.models.classification.resnet import *
from glasses.nn.models.classification.densenet import *
from glasses.nn.models.classification.resnetxt import *

from torchsummary import summary
import torch
import torch.nn as nn

In [7]:
summary(ResNet.resnet50(reduction=1).cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
         Conv2dPad-5          [-1, 256, 56, 56]          16,384
       BatchNorm2d-6          [-1, 256, 56, 56]             512
              ReLU-7          [-1, 256, 56, 56]               0
         Conv2dPad-8          [-1, 256, 28, 28]         589,824
       BatchNorm2d-9          [-1, 256, 28, 28]             512
             ReLU-10          [-1, 256, 28, 28]               0
        Conv2dPad-11          [-1, 256, 28, 28]          65,536
      BatchNorm2d-12          [-1, 256, 28, 28]             512
        Conv2dPad-13          [-1, 256, 28, 28]          16,384
      BatchNorm2d-14          [-1, 256,

(tensor(223439144), tensor(223439144), tensor(852.3527), tensor(1028.2468))

In [ ]:
ResNetBottleneckPreActBlock(64, 256, stride=2)

In [ ]:
FishNetBottleNeck(32, 64)

In [ ]:
ResNetBottleneckPreActBlock(32, 256, shortcut=FishNetShortCut)

In [ ]:
model = ResNet.resnet50()
model

In [ ]:
model = ResNet.resnet50()

summary(model.cuda(), (3,224,224))

In [ ]:
from torchvision.models import resnet50

model = resnet50(False)

summary(model.cuda(), (3,224,224))

In [ ]:
list(map(lambda x: x* 4, [64, 128, 256, 512]))

In [ ]:
FishNet.fishnet99()

In [ ]:
from glasses.nn.models.classification import SpatialSE

block = lambda in_ch, out_ch: nn.Sequential(FishNetBottleNeck(in_ch, out_ch), SpatialSE(out_ch))

block(32, 64)

In [ ]:
FishNet.fishnet99(activation = nn.SELU)
# change number of classes (default is 1000 )
FishNet.fishnet99(n_classes=100)
# pass a different block
block = lambda in_ch, out_ch, **kwargs: nn.Sequential(FishNetBottleNeck(in_ch, out_ch), SpatialSE(out_ch))
FishNet.fishnet99(block=block)